# To export/import an opensearch index, either use directly opensearchpy

In [41]:
import json
from opensearchpy import OpenSearch
from opensearchpy.helpers import bulk


client = OpenSearch(hosts=[{"host": "localhost", "port": 9200}], http_compress=True)

query = {"size": 10000, "query": {"match_all": {}}}
search_result = client.search(index="ind_name", body=query)

with open("ind_name.json", "w") as file:
    json.dump(search_result, file)

client.indices.delete(index="ind_name")

with open("ind_name.json", "r") as file:
    data = json.load(file)

# Prepare bulk indexing data
actions = [
    {
        "_source": doc["_source"],
    }
    for doc in data["hits"]["hits"]
]

bulk(client, actions, index="ind_name")

(406, [])

# Or use awswrangler

In [42]:
# Export and import an index in OpenSearch using AWS Wrangler (python 3.12.3)
# uv add awswrangler[opensearch] ipykernel awscli
# "aws configure" for awswrangler to work
import awswrangler as wr

oc = wr.opensearch.connect(host="localhost", port=9200)
results = wr.opensearch.search(
    oc,
    index="ind_name",
    size=10000,
)
results.to_csv("ind_name.csv", index=False)
wr.opensearch.delete_index(
    oc,
    index="ind_name",
)
wr.opensearch.index_csv(
    oc,
    path="ind_name.csv",
    index="ind_name",
)

{'success': 406, 'errors': []}